In [3]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('GOOGLE_API_KEY')

llm = GooglePalm(google_api_key=api_key, temperature=0.2)
poem = llm("Write a 4 sentence poem about dogs")
print(poem)

Dogs are loyal, loving creatures,
Always there to greet us with a wagging tail.
They make us laugh, they make us smile,
And they fill our hearts with joy.
We are lucky to have dogs in our lives.


In [4]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = os.getenv('DB_PASSWORD')
db_host = "localhost"
db_name = "tshirt_store"


db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	tshirt_id INTEGER, 
	discount_percentage DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(tshirt_id) REFERENCES tshirts (tshirt_id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	tshirt_id	discount_percentage
1	1	5.96
2	2	24.36
3	3	23.91
*/


CREATE TABLE tshirts (
	tshirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand VARCHAR(50), 
	color VARCHAR(20), 
	size VARCHAR(10), 
	price DECIMAL(10, 2), 
	stock_quantity INTEGER, 
	PRIMARY KEY (tshirt_id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from tshirts table:
tshirt_id	brand	color	size	price	stock_quantity
1	nike	blue	m	66.65	100
2	nike	red	l	52.70	50
3	puma	black	xs	79.46	102
*/


In [6]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True) # verbose=True -> Shows the thought process of the LLM
ans1 = db_chain.run("How many tshirts do we have left in total for adidas in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many tshirts do we have left in total for adidas in extra small size and white color?
SQLQuery:SELECT SUM(stock_quantity) FROM tshirts WHERE brand = 'adidas' AND size = 'xs' AND color = 'white'
SQLResult: [(Decimal('180'),)]
Answer:180
> Finished chain.


In [7]:
ans2 = db_chain.run("How much is the price of the inventory for all small size tshirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size tshirts?
SQLQuery:SELECT SUM(price) FROM tshirts WHERE size = 'xs'
SQLResult: [(Decimal('1348.87'),)]
Answer:1348.87
> Finished chain.


The answer is wrong. Because the llm thinks the 'price' column is the total price, instead of price per unit.
We will later fix this with few-shot prompting.

In [8]:
ans2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM tshirt_store.tshirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM tshirt_store.tshirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM tshirt_store.tshirts WHERE size = 'S'
SQLResult: [(Decimal('116903.19'),)]
Answer:116903.19
> Finished chain.


In [9]:
ans3 = db_chain.run("If we have to sell all the Nike's tshirts today with discounts applied, how much revenue would our store generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike's tshirts today with discounts applied, how much revenue would our store generate (post discounts)?
SQLQuery:SELECT SUM(price*(1-discount_percentage)) FROM tshirts AS t JOIN discounts AS d ON t.tshirt_id = d.tshirt_id WHERE brand = 'nike' AND CURDATE() BETWEEN d.start_date AND d.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'd.start_date' in 'where clause'")
[SQL: SELECT SUM(price*(1-discount_percentage)) FROM tshirts AS t JOIN discounts AS d ON t.tshirt_id = d.tshirt_id WHERE brand = 'nike' AND CURDATE() BETWEEN d.start_date AND d.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

It returned a wrong query because the llm thinks the table has start and end dates, but it does not. We will later solve this problem with few shot learning. Lets run the query direclty.

In [10]:
query = """
select sum(a.total_amount * ((100-COALESCE(discounts.discount_percentage,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, tshirt_id from tshirts where brand = 'nike'
group by tshirt_id) a left join discounts on a.tshirt_id = discounts.tshirt_id
"""

ans3 = db_chain.run(query)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.discount_percentage,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, tshirt_id from tshirts where brand = 'nike'
group by tshirt_id) a left join discounts on a.tshirt_id = discounts.tshirt_id

SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.discount_percentage,0))/100)) AS total_revenue
FROM
(SELECT sum(price*stock_quantity) AS total_amount, tshirt_id
FROM tshirts
WHERE brand = 'nike'
GROUP BY tshirt_id) a
LEFT JOIN discounts ON a.tshirt_id = discounts.tshirt_id
SQLResult: [(Decimal('111278.67355000'),)]
Answer:111278.67355
> Finished chain.


In [11]:
ans4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'skechers'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'skechers'
SQLQuery:SELECT SUM(price * stock_quantity) FROM tshirts WHERE brand = 'skechers'
SQLResult: [(Decimal('174296.23'),)]
Answer:174296.23
> Finished chain.


In [12]:
ans5 = db_chain.run("How many white Adidas tshirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white Adidas tshirts we have available?
SQLQuery:SELECT stock_quantity FROM tshirts WHERE brand = 'Adidas' AND color = 'white'
SQLResult: [(112,), (16,), (125,), (115,), (180,), (107,), (26,), (52,), (170,)]
Answer:112
> Finished chain.


This is again wrong. The model needed to use SUM(stock_quantity). Lets run the query itself.

In [13]:
ans5 = db_chain.run("SELECT sum(stock_quantity) FROM tshirt_store.tshirts WHERE brand='nike' AND color='white';")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM tshirt_store.tshirts WHERE brand='nike' AND color='white';
SQLQuery:SELECT sum(stock_quantity) FROM tshirt_store.tshirts WHERE brand='nike' AND color='white';
SQLResult: [(Decimal('49'),)]
Answer:49
> Finished chain.


**FEW SHOT LEARNING**

Now we will use few shot learning to solve the problems we have.

In [14]:
# Lets create some examples to help the LLM understand better
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : ans1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': ans2},
    {'Question': "If we have to sell all the nike tshirts today with discounts applied. How much revenue would we generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.discount_percentage,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, tshirt_id from tshirts where brand = 'nike'
group by tshirt_id) a left join discounts on a.tshirt_id = discounts.tshirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': ans3} ,
     {'Question' : "If we have to sell all the skechers' T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'skechers'",
      'SQLResult': "Result of the SQL query",
      'Answer' : ans4},
    {'Question': "How many white color adidas tshirts do I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'adidas' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : ans5
     }
]

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# We are basically transforming sentences into vectors, which is called embedding.
e = embeddings.embed_query("How many white color adidas tshirts do I have?")

In [20]:
e[:5] # First five elements of the embedding

[0.04511164501309395,
 0.01694350130856037,
 -0.01319942343980074,
 -0.015828102827072144,
 0.11818007379770279]

In [21]:
# Lets create a string for each question and its query, to vectorize them later
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 180",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 116903.19",
 "If we have to sell all the nike tshirts today with discounts applied. How much revenue would we generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.discount_percentage,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, tshirt_id from tshirts where brand = 'nike'\ngroup by tshirt_id) a left join discounts on a.tshirt_id = discounts.tshirt_id\n  Result of the SQL query 111278.67355",
 "If we have to sell all the skechers' T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 's

In [22]:
from langchain.vectorstores import Chroma

# Create a vector database
vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [23]:
# Now when we get a new question, we need to get a similar example from our database
# SemanticSimilarityExampleSelector does this exactly.
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2 # get 2 examples
)
# Lets try an example. It should give us 2 examples that is similar to our question (query)
example_selector.select_examples({"Question": "How many Adidas tshirts I have left in my store?"})

[{'Answer': '49',
  'Question': 'How many white color adidas tshirts do I have?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'adidas' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '180',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'}]

In [24]:
# The LLM can sometimes query columns that do not exist in our database. To solve problems like this, we will use a prompt from langchain
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
print(_mysql_prompt)
print(PROMPT_SUFFIX)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [25]:
# Create a prompt template for an example prompt
from langchain.prompts.prompt import PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [26]:
# Our actual query will come in between the '_mysql_prompt' and PROMPT_SUFFIX. Which will form the latest message that will be sent to the LLM.
# Lets create a template to do that.
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector= example_selector, # If the LLM gets confused, we tell it to look into our database for examples
    example_prompt= example_prompt,
    prefix= _mysql_prompt,
    suffix= PROMPT_SUFFIX,
    input_variables= ["input", "table_info", "top_k"]
)

In [27]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt= few_shot_prompt)

In [29]:
new_chain("How many white color Nike tshirts do we have?")



> Entering new SQLDatabaseChain chain...
How many white color Nike tshirts do we have?
SQLQuery:SELECT sum(stock_quantity) FROM tshirts WHERE brand = 'Nike' AND color = 'White'
SQLResult: [(Decimal('49'),)]
Answer:49
> Finished chain.


{'query': 'How many white color Nike tshirts do we have?', 'result': '49'}

In [30]:
new_chain("How much is the price of the inventory for all small sized tshirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small sized tshirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM tshirts WHERE size = 'S'
SQLResult: [(Decimal('116903.19'),)]
Answer:116903.19
> Finished chain.


{'query': 'How much is the price of the inventory for all small sized tshirts?',
 'result': '116903.19'}

In [31]:
new_chain("If we have to sell all the nike tshirts today with discounts applied. How much revenue would we generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the nike tshirts today with discounts applied. How much revenue would we generate (post discounts)?
SQLQuery:SELECT sum(price * stock_quantity * (100 - discount_percentage) / 100) FROM tshirts JOIN discounts ON tshirts.tshirt_id = discounts.tshirt_id WHERE brand = 'nike'
SQLResult: [(Decimal('19409.23355000'),)]
Answer:19409.23355
> Finished chain.


{'query': 'If we have to sell all the nike tshirts today with discounts applied. How much revenue would we generate (post discounts)?',
 'result': '19409.23355'}